In [0]:
from keras.utils import to_categorical
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Concatenate, GlobalMaxPooling1D
from keras import layers
import keras
import keras.utils
from keras import utils as np_utils
import numpy as np
import itertools
from keras.callbacks import ModelCheckpoint

y_tr_data = []
y_tst_data = []
f = open('drive/colab/y_tr.txt', 'r', encoding = 'utf-8')
for l in f:
  l = l.split()
  y_tr_data.append(int(l[0]))
f.close()
f = open('drive/colab/y_tst.txt', 'r', encoding = 'utf-8')
for l in f:
  l = l.split()
  y_tst_data.append(int(l[0]))
f.close()

max_len = 60
vecto_len = 300
len_data = len(y_tr_data) + len(y_tst_data)

for i in range(len(y_tr_data)):
    y_tr_data[i] = y_tr_data[i] - 1
for j in range(len(y_tst_data)):
    y_tst_data[j] = y_tst_data[j] - 1

classes = np.max(y_tr_data)+1
y_tr_data = to_categorical(y_tr_data,classes)
y_tst_data = to_categorical(y_tst_data,classes)

f3 = open('drive/colab/x_train_300.txt','r',encoding='utf-8')
x_tr = [[[0 for i in range(vecto_len)]for j in range (max_len)] for k in range(len(y_tr_data))]
k = 0
j = 0
for line in f3:
    line = line.split()
    temp = [float(l) for l in line]
    for i in range(vecto_len):
        x_tr[k][j][i] = temp[i]
    j = j+1
    if j == max_len:
        k = k+1
        j = 0
f3.close()

f4 = open('drive/colab/x_test_300.txt', 'r', encoding='utf-8')
x_tst = [[[0 for i in range(vecto_len)] for j in range(max_len)] for k in range(len(y_tst_data))]
k = 0
j = 0
for line in f4:
      line = line.split()
      temp = [float(l) for l in line]
      for i in range(vecto_len):
          x_tst[k][j][i] = temp[i]
      j = j + 1
      if j == max_len:
          k = k + 1
          j = 0
f4.close()

x_tr = np.array(x_tr)
x_tst = np.array(x_tst)
y_tr_data = np.array(y_tr_data)
y_tst_data = np.array(y_tst_data)

x_input = Input(shape=(max_len, vecto_len),dtype='float')
x = Conv1D(filters= 256, kernel_size= 2,activation='relu')(x_input)
x = GlobalMaxPooling1D()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.05)(x)
out = Dense(units=classes,activation='softmax')(x)
model = Model(inputs=x_input,outputs= out)

model.compile(loss='mean_squared_error',optimizer='adam',metrics=['acc'])
batch_size = 128
epochs = 50
model.fit(x_tr, y_tr_data, batch_size=batch_size, epochs = epochs, verbose = 1)
score = model.evaluate(x_tst,y_tst_data, verbose=0)
print('Test accuracy:', score[1])